<a href="https://colab.research.google.com/github/RicardoMiles/Dissertation-template/blob/main/best-random-seed-202411221130.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/SummerProject/src')
!pwd

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/SummerProject/src


In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 定义包含CSV文件的文件夹路径
current_dir = os.getcwd()
root_dir = os.path.dirname(current_dir)
data_dir = os.path.join(root_dir, 'Data')
left_data = os.path.join(data_dir,'left')
right_data = os.path.join(data_dir,'right')

folder_path = left_data

# 获取left文件夹中所有CSV文件的列表
csv_files_left = [os.path.join(left_data, file) for file in os.listdir(left_data) if file.endswith('.csv')]
# 获取right文件夹中所有CSV文件的列表
csv_files_right = [os.path.join(right_data, file) for file in os.listdir(right_data) if file.endswith('.csv')]

print(f"Number of left files: {len(csv_files_left)}")
print(f"Number of right files: {len(csv_files_right)}")

Number of left files: 26
Number of right files: 26


In [3]:
from scipy import stats
# 定义特征提取函数
def extract_features(group):
    features = []
    # 遍历第三列到最后一列
    for col in group.columns[2:]:  # 从第三列开始提取特征
        data = group[col].values
        features.append(data.mean())  # 均值
        features.append(data.std())   # 标准差
        features.append(stats.skew(data))  # 偏度
        features.append(stats.kurtosis(data))  # 峰度
    return features

In [72]:
# 初始化存储所有特征和标签的列表
all_features = []
all_labels = []

# 初始化CSV文件计数器
csv_files_count = 0

# 处理left文件夹中的CSV文件
for file_name in csv_files_left:
    df = pd.read_csv(file_name)
    csv_files_count += 1  # 增加文件计数
    # 将数据按照epoch分组
    grouped = df.groupby(df.columns[1])  # 第二列是epoch分组的列
    for name, group in grouped:
        # 过滤掉time小于0的数据
        group = group[group[df.columns[0]] >= 0]  # 假设time列是第一列
        if len(group) == 81:  # 确保过滤后有81条数据
        # if len(group) == 101:
            # 提取特征
            features = group.iloc[:, 2:].values  # 第三列及以后是EEG通道信号
            all_features.append(features)
            # 获取标签
            all_labels.append('left')

# 处理right文件夹中的CSV文件
for file_name in csv_files_right:
    df_right = pd.read_csv(file_name)
    csv_files_count += 1  # 增加文件计数
    # 将数据按照epoch分组
    grouped_right = df_right.groupby(df_right.columns[1])  # 第二列是epoch分组的列
    for name, group in grouped_right:
        # 过滤掉time小于0的数据
        group = group[group[df_right.columns[0]] >= 0]  # 假设time列是第一列
        if len(group) == 81:  # 确保过滤后有81条数据
        # if len(group) == 101:
            # 提取特征
            features = group.iloc[:, 2:].values  # 第三列及以后是EEG通道信号
            all_features.append(features)
            # 获取标签
            all_labels.append('right')

# 将特征和标签转换为NumPy数组
features_array = np.array(all_features)
labels_array = np.array(all_labels)

# 打印处理的CSV文件数量
print(f"Total CSV files processed: {csv_files_count}")

Total CSV files processed: 52


In [106]:
import numpy as np
from sklearn.model_selection import train_test_split

# 定义衡量分布相似性的函数
def evaluate_split_similarity(X_train, X_test):
    train_mean = X_train.mean(axis=0)
    test_mean = X_test.mean(axis=0)
    # 使用均方差衡量分布差异
    return np.mean((train_mean - test_mean) ** 2)

best_seed = None
min_difference = float('inf')

for seed in range(100):  # 遍历多个种子
    # 将 X 和 y 替换为 features_array 和 labels_array
    X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.2, random_state=seed)
    diff = evaluate_split_similarity(X_train, X_test)
    if diff < min_difference:
        min_difference = diff
        best_seed = seed

print(f"Best seed: {best_seed}, Minimum difference: {min_difference}")


Best seed: 76, Minimum difference: 0.10239083541940588


In [73]:
print(f"Total samples: {features_array.shape[0]}")  # 样本数量
print(f"Timepoints per sample: {features_array.shape[1]}")  # 每个样本的时间点
print(f"Features per timepoint: {features_array.shape[2]}")  # 每个时间点的特征


Total samples: 7220
Timepoints per sample: 81
Features per timepoint: 101


In [99]:
from datetime import datetime
import tensorflow as tf

np.random.seed(76)
tf.random.set_seed(76)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.2, random_state=42)

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

Current processing time: 2024-11-22 11:00:06


In [107]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 初始化标签编码器
label_encoder = LabelEncoder()

# 将训练集和测试集的标签转换为数值
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# 确保转换后的标签是正确的数据类型
y_train_encoded = y_train_encoded.astype('int32')
y_test_encoded = y_test_encoded.astype('int32')

In [108]:
mean_diff = np.abs(X_train.mean(axis=0) - X_test.mean(axis=0))
std_diff = np.abs(X_train.std(axis=0) - X_test.std(axis=0))

print("Mean differences:", mean_diff)
print("Standard deviation differences:", std_diff)


Mean differences: [[0.00494896 0.06979493 0.01380653 ... 0.10897865 0.25091773 0.2856204 ]
 [0.01058503 0.21372214 0.09645687 ... 0.22200588 0.44647853 0.33648155]
 [0.02007572 0.17070905 0.07755432 ... 0.2369642  0.30500684 0.31721984]
 ...
 [0.26159897 0.16135301 0.28502598 ... 0.06859682 0.00128313 0.66405051]
 [0.34033419 0.3832496  0.4039454  ... 0.23834821 0.08901367 0.63715839]
 [0.20053709 0.30466161 0.40744763 ... 0.15510165 0.12940873 0.58545075]]
Standard deviation differences: [[0.64418701 0.33694235 0.05426081 ... 0.26667204 0.65031718 0.13906142]
 [0.29289635 0.39633019 0.02146558 ... 0.15746178 1.23274535 0.18601552]
 [0.08530259 0.46604326 0.28750975 ... 0.17477154 1.47610083 0.21298004]
 ...
 [0.27638451 0.27564424 1.42822097 ... 1.38350335 0.39983428 0.35556301]
 [0.32374998 0.29751529 1.54102984 ... 1.36953322 0.6589136  0.44557057]
 [0.31038196 0.18373149 1.59979472 ... 1.32184636 0.75260442 0.59578401]]


In [109]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, InputLayer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

# 假设 X_train, X_test, y_train, y_test 已经定义并准备好了
# 构建LSTM模型
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.3))  # 添加Dropout层
model.add(LSTM(units=50))
model.add(Dropout(0.3))  # 添加Dropout层
model.add(Dense(1, activation='sigmoid'))  # 二分类问题

# 编译模型
model.compile(Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# 早停法
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# 训练模型
model.fit(X_train, y_train_encoded, epochs=35, batch_size=32, validation_data=(X_test, y_test_encoded))

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5068 - loss: 0.7061 - val_accuracy: 0.5097 - val_loss: 0.7013
Epoch 2/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5281 - loss: 0.6937 - val_accuracy: 0.5104 - val_loss: 0.7007
Epoch 3/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5404 - loss: 0.6899 - val_accuracy: 0.5166 - val_loss: 0.6997
Epoch 4/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5612 - loss: 0.6840 - val_accuracy: 0.5229 - val_loss: 0.6985
Epoch 5/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5558 - loss: 0.6826 - val_accuracy: 0.5242 - val_loss: 0.6997
Epoch 6/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5751 - loss: 0.6755 - val_accuracy: 0.5298 - val_loss: 0.7008
Epoch 7/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5955 - loss: 0.6670 - val_accuracy: 0.5235 - val_loss: 0.7039
Epoch 8/35
181/181 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6007 - loss: 0.6666 - val_accuracy: 

In [110]:
# 预测测试集
predictions = model.predict(X_test)

# 将概率转换为类别
predicted_classes = (predictions > 0.5).astype("int32")

# 打印预测结果
print(predicted_classes)

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
[[0]
 [1]
 [0]
 ...
 [0]
 [1]
 [1]]
Current processing time: 2024-11-22 11:20:30


In [111]:
# 计算精确度、召回率和F1分数
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

precision = precision_score(y_test_encoded, predicted_classes)
recall = recall_score(y_test_encoded, predicted_classes)
f1 = f1_score(y_test_encoded, predicted_classes)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# 计算混淆矩阵
cm = confusion_matrix(y_test_encoded, predicted_classes)
print("Confusion Matrix:")
print(cm)

# 打印当前处理的时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Current processing time: {current_time}")

Precision: 0.49093264248704666
Recall: 0.5383522727272727
F1 Score: 0.513550135501355
Confusion Matrix:
[[347 393]
 [325 379]]
Current processing time: 2024-11-22 11:20:30
